In [2]:
%load_ext autoreload
%autoreload 2

import openai

import re
import glob
import os
import json
import dotenv
dotenv.load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

from textflint import Engine
from textflint.adapter import Config, auto_config
from textflint.input.dataset import Dataset

from models.custom_models import OpenAIFlint, OpenAI_NLI_Flint, OpenAI_MRC_Plus_Flint

In [44]:
dataset = []

with open('./data/reclor.json') as file:
    for line in file:
        dataset.append(json.loads(line))

import random
random.seed(742)
dataset = random.sample(dataset, 1000)

[autoreload of models.custom_models failed: Traceback (most recent call last):
  File "/home/jupyter/conda_env/nlp/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 273, in check
    superreload(m, reload, self.old_objects)
  File "/home/jupyter/conda_env/nlp/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 496, in superreload
    update_generic(old_obj, new_obj)
  File "/home/jupyter/conda_env/nlp/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 393, in update_generic
    update(a, b)
  File "/home/jupyter/conda_env/nlp/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 345, in update_class
    if update_generic(old_obj, new_obj):
  File "/home/jupyter/conda_env/nlp/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 393, in update_generic
    update(a, b)
  File "/home/jupyter/conda_env/nlp/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 305, in update_function
    setat

In [45]:
model = OpenAI_MRC_Plus_Flint('gpt-3.5-turbo', batch_size=64)
results = model.evaluate(dataset, prefix='ori')

Invalid model response: "Not relevant.". The prompt was: {'context': 'In the country of Boldavia at present, only 1 percent of 80-year-olds are left-handed, whereas 10 percent of 50-year-olds and 15 percent of 20-year-olds are lefthanded. Yet over the past 80 years, the proportion of Boldavians who are born left-handed has not increased, nor have Boldavian attitudes toward lefthandedness undergone any important changes.', 'question': 'Which one of the following, if true, most helps to explain the variation in incidence of left-handedness among Boldavians?', 'answers': ['In Boldavia, where men have a shorter average life expectancy than do women, lefthandedness is less common among women than among men.'], 'answer_choices': ['In Boldavia, left-handed people are no more likely than right-handed people to be involved in accidents', 'Ambidexterity is highly valued in Boldavia.', 'Left-handed people have never accounted for more than 18 percent of the population of Boldavia.', 'In Boldavia,

In [47]:
from textflint.input.model.metrics.metrics import accuracy_score as Accuracy

def get_output(content):
    if content[0].isdigit(): return int(content[0])
    elif content[1].isdigit(): return int(content[1])
    else:
        # print('invalid response')
        return -1

results = {}
for k in model.responses.keys():
    contents = [response.choices[0].message.content for response in model.responses[k]]
    outputs = [get_output(c) for c in contents]
    
    labels = [obj['label'] for obj in dataset]
    results[k] = Accuracy(outputs, labels)

In [48]:
results

{'ori': 0.249}

In [3]:
prompt1 = {"hypothesis": "The church has cracks in the ceiling.", "premise": "This church choir sings to the masses as they sing joyous songs from the book at a church.", "y": "neutral", "sample_id": 0}
prompt2 = {"hypothesis": "A man poses in front of an ad.", "premise": "An old man with a package poses in front of an advertisement.", "y": "entailment", "sample_id": 6}

messages=[
    {"role": "system", "content": "You are performing Natural Language Inference. When given a premise and hypothesis, respond with \"entailment\", \"neutral\", or \"contradiction\" and nothing else."},
    {"role": "user", "content": f"Premise: {prompt1['premise']}\nHypothesis: {prompt1['hypothesis']}"},
    {"role": "assistant", "content": f"neutral"},
    {"role": "user", "content": f"Premise: {prompt2['premise']}\nHypothesis: {prompt2['hypothesis']}"}
]

response = openai.ChatCompletion.create(
    model='gpt-3.5-turbo',
    messages=messages,
    temperature=0
)

In [4]:
response

<OpenAIObject chat.completion id=chatcmpl-7Gs9eUhQXyr1O5s36ytjUXIjgabsl at 0x7ff1812f1d50> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "neutral",
        "role": "assistant"
      }
    }
  ],
  "created": 1684255774,
  "id": "chatcmpl-7Gs9eUhQXyr1O5s36ytjUXIjgabsl",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 1,
    "prompt_tokens": 83,
    "total_tokens": 84
  }
}